In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
import os

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def pad_sequence_data_2024(sequences, targets, sequence_lengths_2024, pad_length=87):
    # sequence_tensors = [torch.tensor(seq, dtype=torch.float32) for seq in sequences]
    # target_tensors = [torch.tensor(tgt, dtype=torch.float32) for tgt in targets]
    # padded_sequences = pad_sequence(sequence_tensors, batch_first=True)
    # padded_targets = pad_sequence(target_tensors, batch_first=True)
    # return padded_sequences, padded_targets, sequence_lengths_2024

    sequence_tensors = [torch.tensor(seq, dtype=torch.float32) for seq in sequences]
    target_tensors = [torch.tensor(tgt, dtype=torch.float32) for tgt in targets]
    
    # Manually pad or truncate each sequence to the fixed length
    padded_sequences = []
    padded_targets = []
    
    for seq, tgt in zip(sequence_tensors, target_tensors):
        # Pad sequences (if shorter than pad_length) or truncate them (if longer)
        if seq.size(0) < pad_length:
            padded_seq = F.pad(seq, (0, 0, 0, pad_length - seq.size(0)))  # Pad along the time dimension
        else:
            padded_seq = seq[:pad_length]  # Truncate if it's longer than pad_length
            
        # Pad targets similarly (only time dimension, since targets are 1D)
        if tgt.size(0) < pad_length:
            padded_tgt = F.pad(tgt, (0, pad_length - tgt.size(0)))  # Only pad the time dimension
        else:
            padded_tgt = tgt[:pad_length]  # Truncate if it's longer
            
        padded_sequences.append(padded_seq)
        padded_targets.append(padded_tgt)
    
    # Stack the padded sequences to create batch tensors
    padded_sequences = torch.stack(padded_sequences, dim=0)
    padded_targets = torch.stack(padded_targets, dim=0)
    
    return padded_sequences, padded_targets, sequence_lengths_2024


def evaluate_2024_season(model):
    # Step 1: Load the full dataset for the 2024 season
    combined_df_2024 = pd.read_csv('laptimestest3.csv')  # Load the full dataset
    combined_df_2024 = combined_df_2024.sort_values(by=['raceId', 'driverId', 'lap']) 

    # Select only the data for the 2024 season (all rounds)
    combined_df_2024_season = combined_df_2024[combined_df_2024['year'] == 2024]

    # Apply the same scaling as done on the training data (ensure it's using the same scaler)
    # combined_df_2023_season[features] = feature_scaler.transform(combined_df_2023_season[features])
    # combined_df_2023_season[target] = target_scaler.transform(combined_df_2023_season[[target]])

    combined_df_2024_season = custom_scaler(combined_df_2024_season)

    # Step 2: Form sequences for all rounds in 2024
    sequences_2024 = []
    targets_2024 = []  # Store the true target values
    sequence_lengths_2024 = []

    for (raceId, driverId), group in combined_df_2024_season.groupby(['raceId', 'driverId']):
        group = group.sort_values(by='lap')  # sort by lap within the group
        seq_data = group[features].values  # extract feature columns
        seq_target = group[target].values  # extract target values (true lap times)
        sequences_2024.append(seq_data)
        targets_2024.append(seq_target)
        sequence_lengths_2024.append(len(seq_data))  # Store the sequence lengths

    padded_sequences_2024, padded_targets_2024, sequence_lengths_2024 = pad_sequence_data_2024(sequences_2024, targets_2024, sequence_lengths_2024)
    
    test_dataset_2024 = TestRaceDataset(padded_sequences_2024, padded_targets_2024, sequence_lengths_2024)
    test_data_loader_2024 = DataLoader(test_dataset_2024, batch_size=32, shuffle=False)

    
    model.eval()
    all_predictions = []
    total_loss = 0
    criterion = torch.nn.MSELoss()

    with torch.no_grad():  # Disable gradient calculation for inference
        for batch_X, batch_y, batch_lengths in test_data_loader_2024:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            batch_lengths = batch_lengths.to(device)
            
            outputs = model(batch_X, batch_lengths)  # Get the model outputs
            
            # Apply masking to handle padded sequences
            # mask = torch.arange(outputs.size(1))[None, :] < batch_lengths[:, None]
            mask = torch.arange(outputs.size(1), device=batch_lengths.device)[None, :] < batch_lengths[:, None]
            masked_outputs = outputs[mask]
            masked_targets = batch_y[mask]

            # Calculate loss for the current batch
            loss = criterion(masked_outputs, masked_targets)
            total_loss += loss.item()

            all_predictions.append(masked_outputs.cpu().numpy())

    # Step 5: Post-process the predictions and calculate final loss
    # all_predictions = np.concatenate(all_predictions, axis=0)  # Concatenate predictions
    total_loss /= len(test_data_loader_2024)  # Calculate average loss over all batches

    return total_loss


def pad_sequence_data_2024_round(sequences, sequence_lengths_2024):
    sequence_tensors = [torch.tensor(seq, dtype=torch.float32) for seq in sequences]
    padded_sequences = pad_sequence(sequence_tensors, batch_first=True)
    return padded_sequences, sequence_lengths_2024


def predict_round_22(model):

    combined_df_2024 = pd.read_csv('laptimestest3.csv')  # Load the full dataset
    combined_df_2024 = combined_df_2024.sort_values(by=['raceId', 'driverId', 'lap']) 

    # Select round 24 of the 2024 season
    combined_df_2024_round1 = combined_df_2024[(combined_df_2024['year'] == 2024) & (combined_df_2024['round'] == 24)]

    driver_ids = combined_df_2024_round1.groupby(['raceId', 'driverId']).apply(lambda group: group['driverId'].iloc[0]).values


    # Apply the same scaling as done on the training data (ensure it's using the same scaler)

    combined_df_2024_round1 = custom_scaler(combined_df_2024_round1)


    # Step 2: Form sequences for round 1, 2024
    sequences_2024 = []
    sequence_lengths_2024 = []

    for (raceId, driverId), group in combined_df_2024_round1.groupby(['raceId', 'driverId']):
        group = group.sort_values(by='lap')  # sort by lap within the group
        seq_data = group[features].values  # extract feature columns
        sequences_2024.append(seq_data)
        sequence_lengths_2024.append(len(seq_data))  # Store the sequence lengths

    padded_sequences_2024, sequence_lengths_2024 = pad_sequence_data_2024_round(sequences_2024,sequence_lengths_2024)

    test_dataset_2024 = TestRaceDatasetRound(padded_sequences_2024, sequence_lengths_2024)
    # test_data_loader_2024 = DataLoader(test_dataset_2024, batch_size=32, shuffle=False)

    model.eval()  # Set the model to evaluation mode

    all_predictions = []

    print("Outputs")
    with torch.no_grad():
        for i in range(len(test_dataset_2024)):
            batch_X, length = test_dataset_2024[i]
            length = torch.tensor([length], dtype=torch.int64).cpu()  # Convert to 1D tensor

            batch_X = batch_X.to(device)
            length = length.to(device)
            outputs = model(batch_X.unsqueeze(0), length)  # Add batch dimension to X

            # mask = torch.arange(outputs.size(1))[None, :] < length[:, None]

            # masked_outputs = outputs[mask]
            print(f'O: {outputs}')
            all_predictions.append(outputs)

    # Step 4: Post-process the predictions
    all_predictions = torch.cat(all_predictions, dim=0).cpu().numpy()  # Concatenate predictions

    all_predictions_original_scale = all_predictions

    prediction_features = []
    masked_preds = []

    print('Mask')
    print(all_predictions_original_scale[0])
    for i in range(len(test_dataset_2024)):
        prediction_features.append(test_dataset_2024[i][0])
        
        sequence_length = test_dataset_2024[i][1] if isinstance(test_dataset_2024[i][1], torch.Tensor) else torch.tensor(test_dataset_2024[i][1])

        # Generate the mask for valid indices based on sequence length
        mask = torch.arange(len(all_predictions_original_scale[i])) < sequence_length

        # Apply the mask by slicing, since mask is now compatible with the 1D array
        masked_output = all_predictions_original_scale[i][mask.numpy()]

        masked_preds.append(masked_output)


    print('END Mask')

    # Print or return the predictions
    print(all_predictions_original_scale)


    predictions_df = pd.DataFrame(masked_preds)
    

    # Each driver will have all their lap features in one set of consecutive rows
    prediction_features_array = np.concatenate([np.array(driver_laps) for driver_laps in prediction_features], axis=0)

    # prediction_features_df = pd.DataFrame(prediction_features_array, columns=features)


    driverinfo = pd.read_csv('../../f1db_csv/drivers.csv')

    driver_id_name = []
    for id in driver_ids:
        name = driverinfo.loc[driverinfo['driverId'] == int(id), 'driverRef']
        print(f'{id} Name: {name}')
        if(name.empty):
            name = None
            id = None
        else:
            # id_name = [id, name]
            driver_id_name.append(name.values[0])


    predictions_df.index = driver_id_name

    return predictions_df
    
def custom_scaler(df):
    df_scaled = df.copy()
    
    # Scale specific features
    scaling_factors = {
        'raceId': 100,
        'driverId': 10,
        'constructorId': 10,
        'year': 100,
        'q1milli': 10,
        'q2milli': 10,
        'q3milli': 10,
        'Driver_Season_Points': 10,
        'Races_before': 10,
        'milliseconds_y': 10000
    }
    
    # Apply scaling to main features
    for feature, factor in scaling_factors.items():
        if feature in df_scaled.columns:
            df_scaled[feature] = df_scaled[feature] / factor
    
    # # Apply scaling to P1-P20 prefixed features
    # for i in range(1, 21):
    #     prefix = f'P{i}_'
    #     for feature, factor in scaling_factors.items():
    #         prefixed_feature = f'{prefix}{feature}'
    #         if prefixed_feature in df_scaled.columns:
    #             df_scaled[prefixed_feature] = df_scaled[prefixed_feature] / factor
    
    return df_scaled

def pad_sequence_data(sequences, targets):
    sequence_tensors = [torch.tensor(seq, dtype=torch.float32) for seq in sequences]
    target_tensors = [torch.tensor(tgt, dtype=torch.float32) for tgt in targets]
    padded_sequences = pad_sequence(sequence_tensors, batch_first=True)
    padded_targets = pad_sequence(target_tensors, batch_first=True)
    return padded_sequences, padded_targets, sequence_lengths


def calculate_race_positions(lap_times, mask):
    """
    Calculate total race time and positions for each driver
    lap_times: tensor of shape [batch_size, num_laps]
    mask: tensor of shape [batch_size, num_laps]
    """
    # Sum valid lap times for each driver
    masked_times = lap_times * mask.float()
    total_times = torch.sum(masked_times, dim=1)  # [batch_size]
    
    # Get positions (argsort gives positions in ascending order - fastest first)
    positions = torch.argsort(total_times)
    
    return total_times, positions

def get_driver_features(batch_X, features):
    """
    Extract historical performance features for each driver in the batch
    """
    # Get indices of relevant features
    points_idx = features.index('Driver_Season_Points')
    wins_idx = features.index('driverwins')
    podiums_idx = features.index('Podiums')
    
    # Extract features from first timestep (they remain constant for each sequence)
    driver_points = batch_X[:, 0, points_idx]
    driver_wins = batch_X[:, 0, wins_idx]
    driver_podiums = batch_X[:, 0, podiums_idx]
    
    # Combine features into a single performance score
    historical_performance = (
        driver_points + 
        driver_wins * 25 +  # Weight wins more heavily
        driver_podiums * 15  # Weight podiums less than wins
    )
    
    return historical_performance

def historical_performance_penalty(pred_positions, historical_performance):
    """
    Penalize predictions that deviate from historical performance expectations
    """
    # Get expected positions based on historical performance (higher score -> better position)
    expected_positions = torch.argsort(historical_performance, descending=True)
    
    # Convert to float and normalize to [0, 1] range
    pred_positions_norm = pred_positions.float() / (pred_positions.size(0) - 1)
    expected_positions_norm = expected_positions.float() / (expected_positions.size(0) - 1)
    
    # Calculate MSE between predicted and expected positions
    position_penalty = torch.mean((pred_positions_norm - expected_positions_norm) ** 2)
    
    return position_penalty

def calculate_time_decay_weights(years, current_year=2024, old_decay=0.5, recent_decay=0.1):
    """
    Calculate weights with different decay rates for old and recent data
    old_decay: stronger decay for 2014-2018
    recent_decay: gentler decay for 2019-2022
    """
    years = np.array(years)
    if years.ndim == 0:
        years = np.array([years])
    
    weights = np.zeros_like(years, dtype=float)
    
    for i, year in enumerate(years):
        if (year*100) < 2019:
            # Stronger decay for older years (2014-2018)
            years_from_2018 = 2018 - (year*100)
            weights[i] = np.exp(-old_decay * years_from_2018)
        else:
            # Gentler decay for recent years (2019-2022)
            years_from_current = current_year - (year*100)
            weights[i] = 1.0 * np.exp(-recent_decay * years_from_current)
    
    # Add small constant to prevent zero weights
    weights = weights + 0.01
    
    # Normalize weights
    weights = weights / weights.sum()
    
    return weights[0] if len(weights) == 1 else weights

def combined_loss_function(outputs, targets, mask, batch_X, features, 
                         laptime_weight=0.3, position_weight=0.4, historical_weight=0.3):
    """
    Combine laptime MSE, position, and historical performance losses with time decay
    """
    # Get years and calculate time weights
    years = batch_X[:, 0, features.index('year')].cpu().numpy()
    time_weights = torch.tensor([calculate_time_decay_weights(year) for year in years], 
                              dtype=torch.float32, 
                              device=outputs.device)
    
    # Laptime MSE loss
    masked_outputs = outputs[mask]
    masked_targets = targets[mask]
    laptime_loss = criterion(masked_outputs, masked_targets)
    
    # Position loss
    pred_times, pred_positions = calculate_race_positions(outputs, mask)
    true_times, true_positions = calculate_race_positions(targets, mask)
    position_loss = criterion(pred_positions.float(), true_positions.float())
    
    # Historical performance loss
    historical_performance = get_driver_features(batch_X, features)
    historical_loss = historical_performance_penalty(pred_positions, historical_performance)
    
    # Apply time weights to each loss component
    weighted_laptime_loss = torch.mean(laptime_loss * time_weights)
    weighted_position_loss = torch.mean(position_loss * time_weights)
    weighted_historical_loss = torch.mean(historical_loss * time_weights)
    
    # Combine losses with weights
    total_loss = (
        laptime_weight * weighted_laptime_loss + 
        position_weight * weighted_position_loss +
        historical_weight * weighted_historical_loss
    )
    
    return total_loss, weighted_laptime_loss, weighted_position_loss, weighted_historical_loss
    
# Create a custom PyTorch Dataset
class RaceDataset(Dataset):
    def __init__(self, X, y, lengths):
        self.X = X
        self.y = y
        self.lengths = lengths

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i], self.lengths[i]
    
class LSTMModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.2):
        super(LSTMModel, self).__init__()
        
        # Multiple LSTM layers with dropout
        self.lstm = torch.nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0  # Dropout between LSTM layers
        )
        
        # Dropout layer for the output of the last LSTM layer
        self.dropout = torch.nn.Dropout(dropout)
        
        # Batch normalization layer
        self.batch_norm = torch.nn.BatchNorm1d(hidden_size)
        
        # Fully connected layer
        self.fc = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x, lengths):
        # Pack the padded sequences
        packed_input = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        
        # Pass through LSTM layers
        packed_output, (hn, cn) = self.lstm(packed_input)
        
        # Unpack the output
        output, _ = pad_packed_sequence(packed_output, batch_first=True, total_length=87)
        
        # Apply batch normalization
        output = self.batch_norm(output.transpose(1, 2)).transpose(1, 2)
        
        # Apply dropout
        output = self.dropout(output)
        
        # Apply the fully connected layer
        output = self.fc(output)
        
        # Squeeze the last dimension to match the shape of batch_y
        output = output.squeeze(-1)
        
        return output

class TestRaceDataset(Dataset):
    def __init__(self, X, y, lengths):
        self.X = X
        self.y = y
        self.lengths = lengths

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i], self.lengths[i]
    

class TestRaceDatasetRound(Dataset):
    def __init__(self, X, lengths):
        self.X = X
        self.lengths = lengths

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.lengths[i]




# Convert model_name column to a set for faster lookup (optional but efficient for large datasets)
# existing_model_names = set(results_df['model_name']) if not results_df.empty else set
import glob

models = glob.glob('../test/*.pth')

print(models)
# Process each prediction file
for model_name in models:
    try:
        features = ['raceId','circuitId','driverId','constructorId', 'grid', 'year', 'round', 'lap', 'q1milli', 'q2milli', 'q3milli', 
                                        'Driver_Season_Points', 'driverwins', 'YOB', 'Races_before', 'Races_won', 'Podiums', 'isSafetyCar', 'isSafetyCarPrev', 
                                        'isPitting','tyre_age', 'tyre_compound', 'isVET', 'isZHO', 'isVER', 'isTSU', 'isSTR', 'isMSC', 'isSAR', 'isRIC', 
                                        'isSAI', 'isRUS', 'isPIA', 'isPER', 'isOCO', 'isNOR', 'isMAG', 'isLEC', 'isLAW', 'isLAT', 'isHUL', 'isHAM', 
                                        'isGAS', 'isDEV', 'isBOT', 'isBEA', 'isALO', 'isALB', 'isRBR', 'isFER', 'isMER', 'isALP', 'isMCL', 'isALF', 
                                        'isAST', 'isHAA', 'isATR', 'isWIL']

        target = 'milliseconds_y'

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        model = torch.load(model_name,map_location=torch.device('cpu'))

        model.eval()
                            
        # Test on 2024 season
        round_22_preds = predict_round_22(model)
        model_name = model_name.replace('../test/', '')
        round_22_preds.to_csv(f'./AD24_preds/Pred-{model_name}.csv')
        
        print(f"Processed {model_name}")
        
    except Exception as e:
        print(f"Error processing {model_name}: {str(e)}")



['../test/model_b8_e150_lr0.0001_h512_l1_d0.3_w0.4_0.4_0.2.pth.pth', '../test/model_b16_e150_lr0.0001_h256_l3_d0.3_w0.6_0.3_0.1.pth.pth', '../test/model_b5_e150_lr0.0001_h256_l1_d0.1_w0.6_0.3_0.1.pth.pth', '../test/model_b5_e150_lr0.0001_h256_l1_d0.3_w0.4_0.4_0.2.pth.pth', '../test/model_b16_e150_lr0.0001_h512_l1_d0.1_w0.4_0.4_0.2.pth.pth', '../test/model_b5_e150_lr0.0001_h256_l2_d0.2_w0.4_0.4_0.2.pth.pth', '../test/model_b5_e150_lr0.0001_h256_l2_d0.1_w0.6_0.3_0.1.pth.pth', '../test/model_b8_e150_lr0.0001_h512_l3_d0.1_w0.4_0.4_0.2.pth.pth', '../test/model_b8_e150_lr0.0001_h512_l2_d0.2_w0.6_0.3_0.1.pth.pth', '../test/model_b16_e150_lr0.0001_h512_l1_d0.3_w0.4_0.4_0.2.pth.pth', '../test/model_b8_e150_lr0.0001_h256_l1_d0.2_w0.4_0.4_0.2.pth.pth', '../test/model_b5_e150_lr0.0001_h512_l3_d0.2_w0.4_0.4_0.2.pth.pth', '../test/model_b16_e150_lr0.0001_h256_l2_d0.1_w0.6_0.3_0.1.pth.pth', '../test/model_b16_e150_lr0.0001_h512_l2_d0.1_w0.6_0.3_0.1.pth.pth', '../test/model_b8_e150_lr0.0001_h256_l2_d0

/state/partition1/job-54634702/ipykernel_2280496/1780817838.py:504: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_name,map_location=torch.device('cp

Outputs
O: tensor([[ 9.2802,  9.5839,  9.2075,  8.9935,  8.9105,  8.8749,  8.8577,  8.8484,
          8.8429,  8.8393,  8.8366,  8.8345,  8.8329,  8.8315,  8.8303,  8.8293,
          8.8285,  8.8277,  8.8271,  8.8265,  8.8259,  8.8252,  8.8243,  8.8231,
          8.8217,  8.8201,  8.8184,  8.8165,  8.8146,  8.8125,  8.8104,  8.8081,
          8.8057,  9.6187, 10.5181,  9.1778,  8.9339,  8.8347,  8.7852,  8.7590,
          8.7441,  8.7345,  8.7272,  8.7209,  8.7150,  8.7090,  8.7030,  8.6968,
          8.6905,  8.6841,  8.6776,  8.6710,  8.6643,  8.6575,  8.6504,  8.6431,
          8.6355,  8.6274,  1.2223,  1.2223,  1.2223,  1.2223,  1.2223,  1.2223,
          1.2223,  1.2223,  1.2223,  1.2223,  1.2223,  1.2223,  1.2223,  1.2223,
          1.2223,  1.2223,  1.2223,  1.2223,  1.2223,  1.2223,  1.2223,  1.2223,
          1.2223,  1.2223,  1.2223,  1.2223,  1.2223,  1.2223,  1.2223]])
O: tensor([[10.0294,  9.9969,  9.6204,  9.4318,  9.3588,  9.3247,  9.3061,  9.2947,
          9.2869,  9.

/state/partition1/job-54634702/ipykernel_2280496/1780817838.py:504: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_name,map_location=torch.device('cp

Outputs
O: tensor([[ 10.0964,   9.8127,   9.4581,   9.0975,   9.0378,   9.0308,   9.0164,
           8.9973,   8.9764,   8.9592,   8.9448,   8.9333,   8.9239,   8.9163,
           8.9101,   8.9051,   8.9010,   8.8977,   8.8949,   8.8925,   8.8905,
           8.8886,   8.8869,   8.8852,   8.8835,   8.8818,   8.8801,   8.8783,
           8.8764,   8.8745,   8.8726,   8.8706,   8.8686,   9.3335,  10.6306,
           9.2610,   8.9110,   8.8532,   8.8547,   8.8452,   8.8269,   8.8079,
           8.7912,   8.7770,   8.7647,   8.7542,   8.7451,   8.7373,   8.7307,
           8.7250,   8.7202,   8.7161,   8.7125,   8.7094,   8.7065,   8.7038,
           8.7013,   8.6987, -11.1890, -11.1890, -11.1890, -11.1890, -11.1890,
         -11.1890, -11.1890, -11.1890, -11.1890, -11.1890, -11.1890, -11.1890,
         -11.1890, -11.1890, -11.1890, -11.1890, -11.1890, -11.1890, -11.1890,
         -11.1890, -11.1890, -11.1890, -11.1890, -11.1890, -11.1890, -11.1890,
         -11.1890, -11.1890, -11.1890]])


...

Outputs
O: tensor([[ 10.2841,   9.9426,   9.5084,   9.0806,   9.0331,   8.9715,   8.9441,
           8.9047,   8.8792,   8.8562,   8.8410,   8.8294,   8.8226,   8.8189,
           8.8182,   8.8199,   8.8235,   8.8287,   8.8350,   8.8422,   8.8499,
           8.8581,   8.8665,   8.8751,   8.8836,   8.8921,   8.9005,   8.9088,
           8.9171,   8.9252,   8.9332,   8.9412,   8.9491,   9.2129,  10.6053,
           9.6351,   9.1632,   9.0167,   8.9717,   8.9348,   8.8991,   8.8669,
           8.8410,   8.8205,   8.8049,   8.7932,   8.7850,   8.7795,   8.7765,
           8.7753,   8.7756,   8.7771,   8.7794,   8.7824,   8.7857,   8.7892,
           8.7929,   8.7966, -12.1377, -12.1377, -12.1377, -12.1377, -12.1377,
         -12.1377, -12.1377, -12.1377, -12.1377, -12.1377, -12.1377, -12.1377,
         -12.1377, -12.1377, -12.1377, -12.1377, -12.1377, -12.1377, -12.1377,
         -12.1377, -12.1377, -12.1377, -12.1377, -12.1377, -12.1377, -12.1377,
         -12.1377, -12.1377, -12.1377]])


/state/partition1/job-54634702/ipykernel_2280496/1780817838.py:504: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_name,map_location=torch.device('cp

Outputs
O: tensor([[10.3757,  9.6574,  9.4315,  9.2171,  9.1251,  9.0783,  9.0559,  9.0374,
          9.0230,  9.0114,  9.0032,  8.9979,  8.9960,  8.9972,  9.0013,  9.0076,
          9.0148,  9.0212,  9.0255,  9.0269,  9.0252,  9.0207,  9.0139,  9.0052,
          8.9949,  8.9834,  8.9709,  8.9576,  8.9436,  8.9289,  8.9139,  8.8985,
          8.8829,  9.7737, 10.9383,  9.1241,  8.9727,  8.8874,  8.9066,  8.8972,
          8.8877,  8.8779,  8.8703,  8.8632,  8.8561,  8.8488,  8.8411,  8.8332,
          8.8253,  8.8172,  8.8093,  8.8015,  8.7938,  8.7863,  8.7791,  8.7721,
          8.7654,  8.7590, -1.7075, -1.7075, -1.7075, -1.7075, -1.7075, -1.7075,
         -1.7075, -1.7075, -1.7075, -1.7075, -1.7075, -1.7075, -1.7075, -1.7075,
         -1.7075, -1.7075, -1.7075, -1.7075, -1.7075, -1.7075, -1.7075, -1.7075,
         -1.7075, -1.7075, -1.7075, -1.7075, -1.7075, -1.7075, -1.7075]])
O: tensor([[10.0757,  9.7454,  9.6196,  9.4944,  9.4295,  9.4006,  9.3914,  9.3826,
          9.3751,  9.

/state/partition1/job-54634702/ipykernel_2280496/1780817838.py:504: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_name,map_location=torch.device('cp

Outputs
O: tensor([[10.9983, 11.1131, 10.5132, 10.2160, 10.0674,  9.9826,  9.9293,  9.8934,
          9.8673,  9.8473,  9.8312,  9.8177,  9.8060,  9.7958,  9.7866,  9.7783,
          9.7705,  9.7632,  9.7563,  9.7496,  9.7431,  9.7367,  9.7302,  9.7237,
          9.7171,  9.7104,  9.7035,  9.6964,  9.6891,  9.6816,  9.6739,  9.6659,
          9.6577, 10.7165, 12.1662, 10.3238,  9.9835,  9.8182,  9.7218,  9.6627,
          9.6246,  9.5990,  9.5810,  9.5681,  9.5587,  9.5518,  9.5470,  9.5437,
          9.5418,  9.5409,  9.5409,  9.5417,  9.5429,  9.5446,  9.5466,  9.5488,
          9.5510,  9.5531,  1.1974,  1.1974,  1.1974,  1.1974,  1.1974,  1.1974,
          1.1974,  1.1974,  1.1974,  1.1974,  1.1974,  1.1974,  1.1974,  1.1974,
          1.1974,  1.1974,  1.1974,  1.1974,  1.1974,  1.1974,  1.1974,  1.1974,
          1.1974,  1.1974,  1.1974,  1.1974,  1.1974,  1.1974,  1.1974]])
O: tensor([[11.2294, 11.1824, 10.5778, 10.2936, 10.1518, 10.0700, 10.0178,  9.9815,
          9.9543,  9.

In [2]:
import pandas as pd
import numpy as np
import glob
import os

def analyze_race_predictions(predictions_df):
    
    
    # Get number of laps for each driver
    lap_counts = predictions_df.count(axis=1) 
    
    # Get drivers who completed all laps (Abu Dhabi GP had 58 laps)
    full_race_drivers = lap_counts[lap_counts == 58].index
    
    # Calculate total race time only for drivers who completed all laps
    total_times = predictions_df.loc[full_race_drivers].sum(axis=1)  # Sum along rows

    

    # Sort by total time (ascending)
    predicted_order = total_times.sort_values()

    print(predicted_order.index)
    
    
    
    
    return predicted_order

def evaluate_prediction(predicted_order, model_name):
    
    # Get predicted positions
    predicted_positions = predicted_order.index.tolist()
    
    
    # Check predictions
    correct_winner = predicted_positions[0] == "norris"
    
    podium_matches = len(set(predicted_positions[:3]) & {"norris", "sainz", "leclerc"})
    
    top5_matches = len(set(predicted_positions[:5]) & {"norris", "sainz", "leclerc", "hamilton", "russell"})
    top10_matches = len(set(predicted_positions[:10]) & {"norris", "sainz", "leclerc", "hamilton", "russell","max_verstappen", "gasly", "hulkenberg", "alonso", "piastri"})
    
    return {
        "model": model_name,
        "correct_winner": correct_winner,
        "podium_matches": podium_matches,
        "top5_matches": top5_matches,
        "top10_matches": top10_matches
    }

# List to store all model results
all_results = []

# Get all prediction files
prediction_files = glob.glob('./AD24_preds/Pred-*.csv')

# Process each prediction file
for file_path in prediction_files:
    try:
        # Read predictions
        predictions_df = pd.read_csv(file_path, index_col=0)
        
        # Get model name from file name
        model_name = file_path.replace('./Pred-', '').replace('.csv', '')
        
        # Analyze predictions
        predicted_order = analyze_race_predictions(predictions_df)
        
        # Evaluate predictions
        result = evaluate_prediction(predicted_order, model_name)
        
        # Add to results list
        all_results.append(result)
        
        print(f"Processed {file_path}")
        
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")

# Convert results to DataFrame and save
results_df = pd.DataFrame(all_results)
results_df.to_csv('./model_evaluation_results.csv', index=False)

print(f"Processed {len(all_results)} models")
print("Results saved to ./model_evaluation_results.csv")

Index(['sainz', 'max_verstappen', 'norris', 'russell', 'hamilton', 'leclerc',
       'piastri', 'alonso', 'gasly'],
      dtype='object')
Processed ./AD24_preds/Pred-model_b5_e150_lr0.0001_h512_l1_d0.3_w0.6_0.3_0.1.pth.pth.csv
Index(['norris', 'sainz', 'hamilton', 'russell', 'max_verstappen', 'gasly',
       'piastri', 'alonso', 'leclerc'],
      dtype='object')
Processed ./AD24_preds/Pred-model_b16_e150_lr0.0001_h256_l1_d0.2_w0.4_0.4_0.2.pth.pth.csv
Index(['piastri', 'sainz', 'hamilton', 'max_verstappen', 'gasly', 'norris',
       'leclerc', 'russell', 'alonso'],
      dtype='object')
Processed ./AD24_preds/Pred-model_b5_e150_lr0.0001_h512_l3_d0.1_w0.6_0.3_0.1.pth.pth.csv
Index(['max_verstappen', 'russell', 'hamilton', 'norris', 'sainz', 'leclerc',
       'piastri', 'gasly', 'alonso'],
      dtype='object')
Processed ./AD24_preds/Pred-model_b8_e150_lr0.0001_h512_l3_d0.3_w0.4_0.4_0.2.pth.pth.csv
Index(['hamilton', 'max_verstappen', 'russell', 'norris', 'sainz', 'gasly',
       'alonso'